In [3]:
#init spark
!pip install -q pyspark findspark

import findspark
findspark.init()
import pyspark
findspark.find()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("analytics_data").getOrCreate()


In [4]:
path = "../data/bank-additional/bank-additional-full.csv"
raw_df = (
    spark.read
    .option("header", True)
    .option("inferschema", True)
    .option("sep", ";")
    .csv(path)
)

cols = ['contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays', 'previous']
target_col = 'y'
df = raw_df.select(*(cols + [target_col]))

df.show(10, truncate=False)

+---------+-----+-----------+--------+--------+-----+--------+---+
|contact  |month|day_of_week|duration|campaign|pdays|previous|y  |
+---------+-----+-----------+--------+--------+-----+--------+---+
|telephone|may  |mon        |261     |1       |999  |0       |no |
|telephone|may  |mon        |149     |1       |999  |0       |no |
|telephone|may  |mon        |226     |1       |999  |0       |no |
|telephone|may  |mon        |151     |1       |999  |0       |no |
|telephone|may  |mon        |307     |1       |999  |0       |no |
|telephone|may  |mon        |198     |1       |999  |0       |no |
|telephone|may  |mon        |139     |1       |999  |0       |no |
|telephone|may  |mon        |217     |1       |999  |0       |no |
|telephone|may  |mon        |380     |1       |999  |0       |no |
|telephone|may  |mon        |50      |1       |999  |0       |no |
+---------+-----+-----------+--------+--------+-----+--------+---+
only showing top 10 rows



In [5]:
print("schema df: ")
df.printSchema()
print("__________________________________________")

print("\n So dong df: ")
print(df.count())
print("__________________________________________")


schema df: 
root
 |-- contact: string (nullable = true)
 |-- month: string (nullable = true)
 |-- day_of_week: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- campaign: integer (nullable = true)
 |-- pdays: integer (nullable = true)
 |-- previous: integer (nullable = true)
 |-- y: string (nullable = true)

__________________________________________

 So dong df: 
41188
__________________________________________


In [7]:
print("Checking missing values: ")
total = df.count()

for c in df.columns:
    miss = df.filter(F.col(c).isNull()).count()
    print(f"{c:20s} null = {miss}")

Checking missing values: 
contact              null = 0
month                null = 0
day_of_week          null = 0
duration             null = 0
campaign             null = 0
pdays                null = 0
previous             null = 0
y                    null = 0


### * Tỷ lệ phân bố giữa các biến

In [ ]:
from pyspark.sql import functions as F
cat_cols = ['contact', 'month', 'day_of_week', 'y']
for c in cat_cols:
    fre_val = df.groupBy(c).count().orderBy('count', ascending = False)
 
    total = df.count()
    col_expr = ((F.col('count') / total) * 100).cast("double")
    fre_val.withColumn("ratio (%)", F.round(col_expr, 2)).show()


+---------+-----+---------+
|  contact|count|ratio (%)|
+---------+-----+---------+
| cellular|26144|    63.47|
|telephone|15044|    36.53|
+---------+-----+---------+

+-----+-----+---------+
|month|count|ratio (%)|
+-----+-----+---------+
|  may|13769|    33.43|
|  jul| 7174|    17.42|
|  aug| 6178|     15.0|
|  jun| 5318|    12.91|
|  nov| 4101|     9.96|
|  apr| 2632|     6.39|
|  oct|  718|     1.74|
|  sep|  570|     1.38|
|  mar|  546|     1.33|
|  dec|  182|     0.44|
+-----+-----+---------+

+-----------+-----+---------+
|day_of_week|count|ratio (%)|
+-----------+-----+---------+
|        thu| 8623|    20.94|
|        mon| 8514|    20.67|
|        wed| 8134|    19.75|
|        tue| 8090|    19.64|
|        fri| 7827|     19.0|
+-----------+-----+---------+

+---+-----+---------+
|  y|count|ratio (%)|
+---+-----+---------+
| no|36548|    88.73|
|yes| 4640|    11.27|
+---+-----+---------+



### Nhận xét phân tích tần suất các biến phân loại

#### 1. `contact` — Kênh liên hệ
Tỷ lệ **cellular (63.5%)** cao hơn **telephone (36.5%)**.  
Điều này phản ánh đúng thực tế giai đoạn 2008–2010, khi ngân hàng chuyển dần sang gọi qua **di động** để tiếp cận khách hàng nhanh hơn.  

Khi phân tích hiệu quả chiến dịch, cần kiểm tra xem **tỷ lệ “yes”** giữa hai kênh này có khác biệt đáng kể không.  
Nếu di động có tỷ lệ thành công cao hơn, đó là tín hiệu cho việc **ưu tiên kênh liên hệ** này trong các chiến dịch sau.

---

#### 2. `month` — Tháng gọi
Các tháng có nhiều cuộc gọi nhất:
- **May (33%)**
- **July (17%)**
- **August (15%)**
- **June (13%)**

→ Chiếm hơn **75% tổng số cuộc gọi**.  
Có thể đây là **chiến dịch mùa hè**, khi ngân hàng đẩy mạnh huy động vốn hoặc tung sản phẩm mới.

Các tháng khác (Oct, Sep, Mar, Dec) có tỷ lệ rất nhỏ → thường là **off-campaign** hoặc **chiến dịch thử nghiệm**.  
Nên kiểm tra thêm xem **tỷ lệ “yes” có biến động theo mùa** không; ví dụ: tháng May gọi nhiều nhưng hiệu quả có thể không cao.

---

#### 3. `day_of_week` — Ngày gọi
Phân bố **khá đồng đều (khoảng 19–21%)** mỗi ngày.  
Điều này cho thấy ngân hàng triển khai chiến dịch đều trong tuần, **không tập trung riêng vào đầu hoặc cuối tuần**.

Đây là đặc điểm tốt, giúp **loại bỏ bias theo ngày** khi huấn luyện mô hình.  
Tuy nhiên, vẫn nên xem thử **thứ Sáu** có tỷ lệ “yes” cao hơn không — vì khách hàng cuối tuần có thể tâm lý thoải mái hơn.

---

#### 4. `y` — Biến mục tiêu (kết quả)
Dữ liệu **rất mất cân bằng**:  
- **Yes:** 11.3%  
- **No:** 88.7%

Đây là đặc trưng nổi tiếng của **Bank Marketing Dataset**.  
Hệ quả: nếu huấn luyện mô hình mà **không xử lý imbalance**, mô hình sẽ thiên về dự đoán “no”.

→ Ở bước mô hình hóa, cần **cân bằng lớp** bằng:
- `class_weighting`
- `SMOTE`
- hoặc `resampling`

để đảm bảo mô hình học được tín hiệu thực sự của nhóm “yes”.


In [ ]:
#Kiem tra tan suat phan bo cua cac bien so
num_cols = ['duration', 'campaign', 'pdays', 'previous']

df.select(num_cols).describe().show()

percentiles = df.approxQuantile(num_cols, [0.25, 0.5, 0.75, 0.9, 0.95, 0.99], 0.01)
print(percentiles, "\n")


for i, c in enumerate(num_cols):
    res = dict(zip(["25%", "50%", "75%", "90%", "95%", "99%"], percentiles[i]))
    print(f"Percentile cua {c}: ")
    print(res, "\n")

+-------+------------------+-----------------+-----------------+-------------------+
|summary|          duration|         campaign|            pdays|           previous|
+-------+------------------+-----------------+-----------------+-------------------+
|  count|             41188|            41188|            41188|              41188|
|   mean| 258.2850101971448|2.567592502670681|962.4754540157328|0.17296299893172767|
| stddev|259.27924883646455|2.770013542902331|186.9109073447414|0.49490107983928927|
|    min|                 0|                1|                0|                  0|
|    max|              4918|               56|              999|                  7|
+-------+------------------+-----------------+-----------------+-------------------+

[[102.0, 177.0, 312.0, 526.0, 702.0, 4918.0], [1.0, 2.0, 3.0, 5.0, 6.0, 56.0], [999.0, 999.0, 999.0, 999.0, 999.0, 999.0], [0.0, 0.0, 0.0, 1.0, 1.0, 7.0]] 

Percentile cua duration: 
{'25%': 102.0, '50%': 177.0, '75%': 312.0, '90%': 5

### Nhận xét phân tích các biến số

#### 1. `duration` — Thời lượng cuộc gọi
- Trung bình khoảng **258 giây**, độ lệch chuẩn gần bằng trung bình → **phân bố rất lệch phải (right-skewed)**.  
- **75%** cuộc gọi kéo dài ≤ **312 giây**, nhưng **1% trên cùng** lên tới gần **5000 giây (~82 phút)** → có **outlier mạnh**.  
- Điều này hợp lý, vì đa số khách hàng từ chối sớm; còn các cuộc gọi dài thường đến từ khách hàng có hứng thú hoặc nhân viên thuyết phục lâu.  
- Tuy nhiên, như đã nói ở phần trước, `duration` **không nên dùng để dự đoán trực tiếp**, vì đây là **kết quả sau khi gọi**, không phải thông tin biết trước.  
  → Dùng trong **EDA (phân tích khám phá dữ liệu)** để hiểu hành vi khách hàng là hợp lý.



#### 2. `campaign` — Số lần liên hệ trong chiến dịch hiện tại
- **Trung vị = 2**, **75% ≤ 3**, nhưng **1% cao nhất** lên tới **56 lần** → có những khách hàng bị gọi **hơn 50 lần (!)**  
- Đây là **outlier rõ ràng**, nhưng lại chứa **thông tin hành vi**: chiến dịch đã cố gắng “đuổi theo” khách hàng đó.  
- Khi mô hình hóa, nên:
  - **Giới hạn (clip)** giá trị tối đa ở một ngưỡng hợp lý (ví dụ 10), hoặc  
  - **Log-transform** để giảm ảnh hưởng của các giá trị cực lớn.



#### 3. `pdays` — Số ngày từ lần liên hệ trước
- Tất cả các percentile đều = **999**, cho thấy **phần lớn khách hàng chưa từng được liên hệ trước đây**.  
- Theo mô tả dữ liệu, **999 nghĩa là “no previous contact”**.  
- Biến này ít thông tin trực tiếp, nên khi xử lý có thể:
  - Tạo biến nhị phân mới: `has_contact_before = (pdays != 999)`, hoặc  
  - Giữ nguyên 999 và để mô hình tự học (tùy thuật toán).



#### 4. `previous` — Số lần liên hệ trước chiến dịch hiện tại
- **75% khách hàng chưa từng được liên hệ (0)**, **95% ≤ 1**, chỉ vài người đến **7 lần**.  
- Nghĩa là **đa số là khách hàng mới**.  
- Biến này có **phân bố rất lệch (one-sided)**, nên khi đưa vào mô hình cần:
  - **Chuẩn hóa (scaling)** hoặc  
  - **Binning (phân nhóm)** để mô hình dễ học và ổn định hơn.
---

### 1. Insight về hành vi liên hệ (kênh – thời điểm – kết quả):

#### 1.1 Về kênh liên hệ 
→ Mục tiêu: tìm hiểu kênh và thời điểm nào hiệu quả nhất.
**Câu hỏi khai thác:**

- Kênh cellular có giúp tăng tỷ lệ phản hồi tích cực không?

- Liệu telephone có thể bị loại bỏ hoặc giảm tần suất để tiết kiệm chi phí?

In [69]:
#Tỷ lệ khách hàng "yes" của từng kênh
cellular_total = df.filter(df['contact'] == 'cellular').count()
yes_cellular = df.filter((df['contact'] == 'cellular') & (df['y'] == 'yes')).count()
cellular_ratio = round(yes_cellular / cellular_total * 100, 2)
print(f"Ty le khách hang say yes trong cellular la:{cellular_ratio}% tren tong so {cellular_total} \n")



telephone_total = df.filter(df['contact'] == 'telephone').count()
yes_telephone = df.filter((df['contact'] == 'telephone') & (df['y'] == 'yes')).count()
telephone_ratio = round(yes_telephone / telephone_total * 100, 2)
print(f"ty le  khach hang say yes khi su dung telephone la:{telephone_ratio}% tren tong so {telephone_total} ")



Ty le khách hang say yes trong cellular la:14.74% tren tong so 26144 

ty le  khach hang say yes khi su dung telephone la:5.23% tren tong so 15044 


Phân tích cho thấy **kênh di động (cellular)** có tỷ lệ khách hàng đồng ý gửi tiết kiệm **cao gấp gần 3 lần** so với **điện thoại bàn (telephone)**.

- **Cellular:** 14.74% khách hàng đồng ý trên tổng **26,144 cuộc gọi**.  
- **Telephone:** 5.23% khách hàng đồng ý trên tổng **15,044 cuộc gọi**.

👉 **Kết luận:**  
Kênh **di động** cho thấy **hiệu quả vượt trội**, có thể do khách hàng **dễ tiếp cận hơn** và **phản hồi nhanh hơn**.  
**Đề xuất:** Ngân hàng nên **ưu tiên ngân sách và nhân sự cho kênh di động**, đồng thời **giảm tần suất gọi qua điện thoại bàn** để **tối ưu chi phí và nâng cao tỷ lệ chuyển đổi**.

---

#### 1.2 Về tháng gọi (month)

→ **Mục tiêu:** kiểm tra xem yếu tố **mùa vụ** có ảnh hưởng đến tỷ lệ khách hàng đồng ý (“yes”) hay không.

**Câu hỏi khai thác:**

- Có phải **tháng May** tuy có nhiều cuộc gọi nhưng **tỷ lệ thành công lại thấp**?

- **Tháng March** hoặc **September** có thể là “**thời điểm vàng**” – ít cuộc gọi nhưng **hiệu quả cao hơn**?

- Có **mối quan hệ nào giữa số lượng cuộc gọi và tỷ lệ thành công** (gọi nhiều chưa chắc tốt hơn)?


In [80]:
month_df = (
    df.groupBy('month')
      .agg(
          F.count('*').alias('total'),
          F.sum((F.col('y') == 'yes').cast('int')).alias('yes_count')
      )
      .withColumn('yes_ratio (%)', F.round(F.col('yes_count')/F.col('total')* 100, 2))
      .orderBy('yes_ratio (%)')
)
month_df.show()

+-----+-----+---------+-------------+
|month|total|yes_count|yes_ratio (%)|
+-----+-----+---------+-------------+
|  may|13769|      886|         6.43|
|  jul| 7174|      649|         9.05|
|  nov| 4101|      416|        10.14|
|  jun| 5318|      559|        10.51|
|  aug| 6178|      655|         10.6|
|  apr| 2632|      539|        20.48|
|  oct|  718|      315|        43.87|
|  sep|  570|      256|        44.91|
|  dec|  182|       89|         48.9|
|  mar|  546|      276|        50.55|
+-----+-----+---------+-------------+



Phân tích cho thấy các chiến dịch được triển khai mạnh nhất vào **tháng May, July và August**, nhưng **tỷ lệ thành công lại khá thấp**.  
Ngược lại, những tháng có **ít cuộc gọi** như **March, September, October và December** lại có tỷ lệ “yes” **rất cao** — thậm chí **gấp 5–8 lần so với tháng May**.

- **Tháng May:** 6.43% khách hàng đồng ý trên tổng **13,769 cuộc gọi**.  
- **Tháng August:** 10.6% khách hàng đồng ý trên tổng **6,178 cuộc gọi**.  
- **Tháng April:** 20.48% khách hàng đồng ý trên tổng **2,632 cuộc gọi**.  
- **Tháng March:** 50.55% khách hàng đồng ý trên tổng **546 cuộc gọi**.

👉 **Kết luận:**  
Hiệu quả chiến dịch có **tính mùa vụ rõ rệt**.  
Những tháng ngân hàng **gọi nhiều (đặc biệt là May, July)** lại có **tỷ lệ phản hồi thấp**,  
trong khi các tháng **ít gọi (March, September, October, December)** mang lại **tỷ lệ thành công vượt trội**.

**Đề xuất:**  
Ngân hàng nên **tái phân bổ lịch gọi**, **tăng cường chiến dịch** vào các tháng có hiệu quả cao,  
đồng thời **giảm tần suất** ở các tháng thấp hiệu quả như **May–July** để **nâng cao hiệu suất tổng thể**.

---

#### 1.3 Về ngày trong tuần (day_of_week)

→ **Mục tiêu:** xác định **ngày nào trong tuần** mang lại **tỷ lệ khách hàng đồng ý cao nhất**,  
từ đó hỗ trợ ngân hàng **lên lịch gọi tối ưu** cho đội ngũ tư vấn.

**Câu hỏi khai thác:**

- Liệu khách hàng có xu hướng **đồng ý nhiều hơn vào cuối tuần** (thứ Năm, thứ Sáu) khi **tâm lý thoải mái hơn**?

- Có **sự khác biệt rõ** giữa **đầu tuần** và **cuối tuần** hay không?

- Ngân hàng có thể **ưu tiên gọi vào các ngày “hiệu quả” hơn** để **tăng tỷ lệ chuyển đổi**?


In [93]:
dayOfWeek_df = (
    df.groupBy('day_of_week')
      .agg(
         F.count('*').alias('total'),
         F.sum((F.col('y') == 'yes').cast('int')).alias('yes_count')
      )
      .withColumn('yes_ratio(%)', F.round((F.col('yes_count')/F.col('total') * 100), 2))
      .orderBy('yes_ratio(%)',ascending = False)
)
dayOfWeek_df.show()

+-----------+-----+---------+------------+
|day_of_week|total|yes_count|yes_ratio(%)|
+-----------+-----+---------+------------+
|        thu| 8623|     1045|       12.12|
|        tue| 8090|      953|       11.78|
|        wed| 8134|      949|       11.67|
|        fri| 7827|      846|       10.81|
|        mon| 8514|      847|        9.95|
+-----------+-----+---------+------------+



Phân tích cho thấy **tỷ lệ khách hàng “say yes” cao nhất** rơi vào **thứ Năm (12.12%)**,  
tiếp theo là **thứ Ba (11.78%)** và **thứ Tư (11.67%)**.  
Trong khi đó, **thứ Hai có tỷ lệ thấp nhất (9.95%)** — tức là **đầu tuần khách hàng ít phản hồi tích cực hơn**.

- **Thứ Năm:** 12.12% khách hàng đồng ý trên tổng **8,623 cuộc gọi**.  
- **Thứ Ba:** 11.78% khách hàng đồng ý trên tổng **8,090 cuộc gọi**.  
- **Thứ Hai:** 9.95% khách hàng đồng ý trên tổng **8,514 cuộc gọi**.

👉 **Kết luận:**  
Tỷ lệ phản hồi tích cực có **xu hướng tăng dần về giữa và cuối tuần**, đạt **đỉnh vào thứ Năm**.  
Điều này gợi ý rằng **thời điểm giữa – cuối tuần** là **“khung giờ vàng” để triển khai cuộc gọi**,  
khi khách hàng có **tâm lý thoải mái** và **sẵn sàng tương tác hơn**.

---

### 2: “Tần suất & lịch sử liên hệ.”

#### 2.1. Tần suất gọi trong chiến dịch hiện tại – campaign
→ **Mục tiêu:** xem việc **gọi nhiều hơn trong cùng một chiến dịch** có làm **tăng khả năng khách hàng “yes”** hay không.

**Cần phân tích:**

- So sánh **trung bình số lần gọi (campaign)** giữa nhóm **“yes”** và **“no”**.  
- Tính **tỷ lệ “yes” theo nhóm tần suất gọi**, ví dụ: **1–2**, **3–5**, **>5 lần**.

**Câu hỏi khai thác:**

- Liệu **gọi quá nhiều có phản tác dụng** không?  
- Có tồn tại **ngưỡng “số lần gọi tối ưu”** nào cho chiến dịch?

In [ ]:
# Trung bình số lần gọi giữa hai nhóm yes / no
avg_calls = df.groupBy("y").agg(F.round(F.avg("campaign"), 2).alias("avg_calls"))
avg_calls.show()

# Phân nhóm tần suất gọi: ít - trung bình - nhiều
campaign_gr = (
    df.withColumn(
        'campaign_group',
        F.when(F.col('campaign') <= 2, "low")
        .when(F.col('campaign') <= 5, "medium")
        .otherwise("high")
    )
    .groupBy('campaign_group')
    .agg(
        F.count("*").alias("total"),
        F.sum((F.col("y") == "yes").cast("int")).alias("yes_count")
    )
    .withColumn("yes_ratio (%)", F.round(F.col("yes_count") / F.col("total") * 100, 2))
    .orderBy("campaign_group")
)

campaign_gr.show()

+---+---------+
|  y|avg_calls|
+---+---------+
| no|     2.63|
|yes|     2.05|
+---+---------+

+--------------+-----+---------+-------------+
|campaign_group|total|yes_count|yes_ratio (%)|
+--------------+-----+---------+-------------+
|          high| 3385|      186|         5.49|
|           low|28212|     3511|        12.45|
|        medium| 9591|      943|         9.83|
+--------------+-----+---------+-------------+



Phân tích cho thấy những khách hàng **đồng ý gửi tiết kiệm (“yes”)** chỉ được gọi trung bình **2.05 lần**,  
trong khi nhóm **từ chối (“no”)** được gọi trung bình **2.63 lần**.  
Điều này gợi ý rằng **gọi quá nhiều lần không giúp tăng hiệu quả**, thậm chí **có xu hướng phản tác dụng**.

**Phân nhóm tần suất gọi:**

- **Nhóm ít (≤ 2 lần):** tỷ lệ thành công **12.45%** trên tổng **28,212 cuộc gọi**.  
- **Nhóm trung bình (3–5 lần):** tỷ lệ thành công **9.83%** trên tổng **9,591 cuộc gọi**.  
- **Nhóm cao (>5 lần):** tỷ lệ thành công chỉ **5.49%** trên tổng **3,385 cuộc gọi**.

👉 **Kết luận:**  
Tần suất gọi cao **không làm tăng tỷ lệ đồng ý**; ngược lại, **càng gọi nhiều khách hàng càng ít phản hồi tích cực**.  
Điều này cho thấy **nhiều khách hàng có thể cảm thấy phiền** khi bị liên hệ lặp lại trong cùng chiến dịch.

**Đề xuất:**  
Ngân hàng nên **giới hạn số lần gọi tối đa ở mức 2–3 lần mỗi chiến dịch**,  
và **chuyển trọng tâm sang chất lượng cuộc gọi thay vì tần suất**,  
nhằm **tối ưu hiệu quả và giảm chi phí nhân sự**.

---

#### 2.2. Lịch sử liên hệ – `pdays` và `has_contact_before`

→ **Mục tiêu:** tìm hiểu **tác động của việc tái liên hệ** với khách hàng **đã từng được gọi trước đây**.

**Cần phân tích:**

- So sánh **tỷ lệ “yes”** giữa hai nhóm:  
  - **Đã từng được gọi trước** (`pdays ≠ 999`)  
  - **Chưa từng được gọi** (`pdays = 999`)  
- Kiểm tra xem **khoảng cách giữa hai lần liên hệ** (`pdays` nhỏ → gọi gần đây) có **ảnh hưởng đến tỷ lệ “yes”** hay không.

**Câu hỏi khai thác:**

- Việc **gọi lại** có giúp **tăng cơ hội thành công** không?  
- Nếu có, **sau bao lâu gọi lại** là hợp lý?


In [105]:
# Tạo biến đánh dấu đã từng liên hệ trước đó
df_pdays = df.withColumn("has_contact_before", (F.col("pdays") != 999).cast("int"))
total_count = df_pdays.count()

# Đếm số lượng và tỷ lệ "yes" theo nhóm liên hệ
pdays_stats = (
    df_pdays.groupBy("has_contact_before")
    .agg(
        F.count("*").alias("total"),
        F.sum((F.col("y") == "yes").cast("int")).alias("yes_count")
    )
    .withColumn("yes_ratio (%)", F.round(F.col("yes_count") / F.col("total") * 100, 2))
    .withColumn("group_ratio (%)", F.round(F.col("total") / total_count * 100, 2))
    .orderBy("has_contact_before")
)
pdays_stats.show()

# Chỉ giữ những khách hàng đã từng được liên hệ (pdays != 999)
df_recontacted = df.filter(F.col("pdays") != 999)

# Phân nhóm khoảng cách gọi lại
pdays_grouped = (
    df_recontacted.withColumn(
        "pdays_group",
        F.when(F.col("pdays") <= 5, "≤ 5 ngày")
         .otherwise("> 5 ngày")
    )
    .groupBy("pdays_group")
    .agg(
        F.count("*").alias("total"),
        F.sum((F.col("y") == "yes").cast("int")).alias("yes_count")
    )
    .withColumn("yes_ratio (%)", F.round(F.col("yes_count") / F.col("total") * 100, 2))
    .orderBy("pdays_group")
)
pdays_grouped.show()


+------------------+-----+---------+-------------+---------------+
|has_contact_before|total|yes_count|yes_ratio (%)|group_ratio (%)|
+------------------+-----+---------+-------------+---------------+
|                 0|39673|     3673|         9.26|          96.32|
|                 1| 1515|      967|        63.83|           3.68|
+------------------+-----+---------+-------------+---------------+

+-----------+-----+---------+-------------+
|pdays_group|total|yes_count|yes_ratio (%)|
+-----------+-----+---------+-------------+
|   > 5 ngày|  810|      522|        64.44|
|   ≤ 5 ngày|  705|      445|        63.12|
+-----------+-----+---------+-------------+



Phân tích cho thấy 
- **Khách hàng từng được liên hệ trước đó** có **tỷ lệ đồng ý rất cao (~63.8%)**,  
cao gấp nhiều lần so với nhóm **chưa từng được gọi (9.3%)**.  

- Giữa nhóm **được gọi lại trong vòng 5 ngày** và **sau 5 ngày**, tỷ lệ “yes” gần như **tương đương (63–64%)**.

- Chỉ **3.68%** khách hàng từng được gọi trước đó, nghĩa là **hơn 96%** là lần đầu tiên được liên hệ.  Điều này xác nhận chiến dịch marketing của ngân hàng chủ yếu là **cold call**, không phải **chăm sóc khách hàng cũ**.  

👉 **Kết luận:**  
**Tái liên hệ khách hàng cũ** là một **chiến lược hiệu quả rõ rệt**,  
trong khi **khoảng cách giữa hai lần gọi không ảnh hưởng đáng kể**.

**Đề xuất:**  
Ngân hàng nên **ưu tiên xây dựng chiến dịch follow-up có chọn lọc**  
thay vì **tập trung quá nhiều vào gọi mới (cold call)**.

---

#### 2.3. Số lần liên hệ trong các chiến dịch trước – `previous`

→ **Mục tiêu:** đánh giá xem **lịch sử được gọi nhiều lần trước đây** có giúp **tăng xác suất khách hàng “yes”** trong chiến dịch hiện tại hay không.

**Cần phân tích:**

- Tính **tỷ lệ “yes” theo giá trị `previous`** (0, 1, 2, …).  
- Kiểm tra xem **khách hàng từng được liên hệ 1–2 lần trước đó** có **dễ đồng ý hơn** so với nhóm chưa từng liên hệ không.

**Câu hỏi khai thác:**

- “**Khách hàng quen thuộc**” có thực sự **tiềm năng hơn** không?  
- Có nên **tập trung vào nhóm đã liên hệ nhiều lần trong quá khứ** để tăng hiệu quả chiến dịch?


In [106]:
from pyspark.sql import functions as F

# Tính tỷ lệ yes theo số lần được liên hệ trong các chiến dịch trước
previous_stats = (
    df.groupBy("previous")
      .agg(
          F.count("*").alias("total"),
          F.sum((F.col("y") == "yes").cast("int")).alias("yes_count")
      )
      .withColumn("yes_ratio (%)", F.round(F.col("yes_count") / F.col("total") * 100, 2))
      .orderBy("previous")
)

previous_stats.show()


+--------+-----+---------+-------------+
|previous|total|yes_count|yes_ratio (%)|
+--------+-----+---------+-------------+
|       0|35563|     3141|         8.83|
|       1| 4561|      967|         21.2|
|       2|  754|      350|        46.42|
|       3|  216|      128|        59.26|
|       4|   70|       38|        54.29|
|       5|   18|       13|        72.22|
|       6|    5|        3|         60.0|
|       7|    1|        0|          0.0|
+--------+-----+---------+-------------+



Phân tích cho thấy **tỷ lệ đồng ý tăng rõ rệt** theo **số lần khách hàng từng được liên hệ trong quá khứ**:

- **previous = 0:** 8.83% khách hàng đồng ý.  
- **previous = 1–2:** 21–46%, tăng gấp **2–5 lần**.  
- **previous ≥ 3:** ~55–72%, dù số mẫu ít hơn.

👉 **Kết luận:**  
**Khách hàng đã từng được liên hệ trước đây** là nhóm **có tiềm năng cao**,  
với **xác suất “yes” cao hơn nhiều** so với khách hàng mới.

**Đề xuất:**  
Tập trung **chăm sóc và tái liên hệ nhóm khách hàng quen thuộc**,  
thay vì **dàn trải nguồn lực cho nhóm chưa từng tương tác**.